In [1]:
import torch
import glob
import numpy as np
import imageio
import cv2

import torch.utils.data as data
import torch.nn.functional as F
from torchvision.transforms import Compose

In [ ]:
class BadPixelMetric:
    def __init__(self, threshold=1.25, depth_cap=10):
        self.__threshold = threshold
        self.__depth_cap = depth_cap

    def compute_scale_and_shift(self, prediction, target, mask):
        # system matrix: A = [[a_00, a_01], [a_10, a_11]]
        a_00 = torch.sum(mask * prediction * prediction, (1, 2))
        a_01 = torch.sum(mask * prediction, (1, 2))
        a_11 = torch.sum(mask, (1, 2))

        # right hand side: b = [b_0, b_1]
        b_0 = torch.sum(mask * prediction * target, (1, 2))
        b_1 = torch.sum(mask * target, (1, 2))

        # solution: x = A^-1 . b = [[a_11, -a_01], [-a_10, a_00]] / (a_00 * a_11 - a_01 * a_10) . b
        x_0 = torch.zeros_like(b_0)
        x_1 = torch.zeros_like(b_1)

        det = a_00 * a_11 - a_01 * a_01
        # A needs to be a positive definite matrix.
        valid = det > 0

        x_0[valid] = (a_11[valid] * b_0[valid] - a_01[valid] * b_1[valid]) / det[valid]
        x_1[valid] = (-a_01[valid] * b_0[valid] + a_00[valid] * b_1[valid]) / det[valid]

        return x_0, x_1

    def __call__(self, prediction, target, mask):
        # transform predicted disparity to aligned depth
        target_disparity = torch.zeros_like(target)
        target_disparity[mask == 1] = 1.0 / target[mask == 1]

        scale, shift = self.compute_scale_and_shift(prediction, target_disparity, mask)
        prediction_aligned = scale.view(-1, 1, 1) * prediction + shift.view(-1, 1, 1)

        disparity_cap = 1.0 / self.__depth_cap
        prediction_aligned[prediction_aligned < disparity_cap] = disparity_cap

        prediciton_depth = 1.0 / prediction_aligned

        # bad pixel
        err = torch.zeros_like(prediciton_depth, dtype=torch.float)

        err[mask == 1] = torch.max(
            prediciton_depth[mask == 1] / target[mask == 1],
            target[mask == 1] / prediciton_depth[mask == 1],
        )

        err[mask == 1] = (err[mask == 1] > self.__threshold).float()

        p = torch.sum(err, (1, 2)) / torch.sum(mask, (1, 2))

        return 100 * torch.mean(p)


## File organization

In [1]:
import os

def count_images(directory, extensions={".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}):
    image_count = 0
    
    for root, _, files in os.walk(directory):
        image_count += sum(1 for file in files if os.path.splitext(file)[1].lower() in extensions)
    
    return image_count



In [23]:
# Example usage
directory_path = "/home/gkmo/workspace/data/KITTI_eigen_split/train/rgb"  # Change this to the folder you want to scan
print(f"Total image files: {count_images(directory_path)/2}")

Total image files: 11579.0


In [26]:
# Example usage
directory_path = "/home/gkmo/workspace/data/KITTI_eigen_split/test/output_rgb"  # Change this to the folder you want to scan
print(f"Total image files: {count_images(directory_path)}")

Total image files: 652


In [5]:
import os
import shutil

base_folder = '/home/gkmo/workspace/data/new_splits/val/labels'
depth_folder = '/home/gkmo/workspace/data/Depth_anything_v2/test/depth'
output_folder = '/home/gkmo/workspace/data/new_anything_split/val'


def copy_matching_files(base_folder, depth_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # List all files in the base folder
    base_files = os.listdir(base_folder)

    # Iterate over each file in the base folder
    for file in base_files:
        # Change the extension to .png
        base_name, _ = os.path.splitext(file)
        depth_file = f"{base_name}.png"

        # Construct the full path to the depth file
        depth_file_path = os.path.join(depth_folder, depth_file)

        # Check if the depth file exists
        if os.path.exists(depth_file_path):
            # Copy the depth file to the output folder
            shutil.copy2(depth_file_path, output_folder)
            print(f"Copied: {depth_file_path} to {output_folder}")
        else:
            print(f"Depth file not found: {depth_file_path}")

# Example usage

copy_matching_files(base_folder, depth_folder, output_folder)

Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/000585.png to /home/gkmo/workspace/data/new_anything_split/val
Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/005218.png to /home/gkmo/workspace/data/new_anything_split/val
Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/002522.png to /home/gkmo/workspace/data/new_anything_split/val
Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/002558.png to /home/gkmo/workspace/data/new_anything_split/val
Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/003595.png to /home/gkmo/workspace/data/new_anything_split/val
Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/000622.png to /home/gkmo/workspace/data/new_anything_split/val
Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/000765.png to /home/gkmo/workspace/data/new_anything_split/val
Copied: /home/gkmo/workspace/data/Depth_anything_v2/test/depth/004839.png to /home/gkmo/workspace/data/new_anything_sp

Depth folder does not exist: /home/gkmo/workspace/data/Depth_anything_v2/train/images
Depth folder does not exist: /home/gkmo/workspace/data/Depth_anything_v2/test/images
Depth folder does not exist: /home/gkmo/workspace/data/Depth_anything_v2/val/images
Depth maps copied into /home/gkmo/workspace/data/new_splits/output_depth.

All label files had corresponding depth maps.
